In [ ]:
# All Includes
import random
import numpy as np
import tensorflow as tf
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf 
from sklearn import metrics
import os
import pandas as pd
from collections import Counter
from sklearn.preprocessing import LabelEncoder

# os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
# # physical_devices = tf.config.experimental.list_physical_devices('GPU')
# # tf.config.experimental.set_memory_growth(physical_devices[0], True)
# configuration = tf.compat.v1.ConfigProto()
# configuration.gpu_options.allow_growth = True
# session = tf.compat.v1.Session(config=configuration)


In [ ]:
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   try:
#     # Currently, memory growth needs to be the same across GPUs
#     for gpu in gpus:
#       tf.config.experimental.set_memory_growth(gpu, True)
#     logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#     print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#   except RuntimeError as e:
#     # Memory growth must be set before GPUs have been initialized
#     print(e)

In [ ]:
INPUT_SIGNAL_TYPES = [
    "Accelerometer_x",
    "Accelerometer_y",
    "Accelerometer_z",
    "Gyroscope_x",
    "Gyroscope_y",
    "Gyroscope_z",
    "Gravity_x",
    "Gravity_y",
    "Gravity_z",
    "Magnetometer_x",
    "Magnetometer_y",
    "Magnetometer_z"
]


In [ ]:
def load_data(window_size,position):
#     window_size=0.8
    path="E:/polyu/代码/数据处理代码/new/data_for_dl_category by sheets_"+position+".xlsx_" + str(window_size) + ".csv"
    data=pd.read_csv(path,header=0,index_col=None)
    subject_names=["Rebar1","Rebar2","Rebar3","Carpenter1","Carpenter2","Masonry1","Masonry2"]
    data.loc[data["Label_2"]==4,"Label_2"]=3
    data.loc[data["Label_2"]==5,"Label_2"]=4
    data.loc[data["Label_2"]==6,"Label_2"]=3
    data.loc[data["Label_2"]==7,"Label_2"]=3
    data.loc[data["Label_2"]==8,"Label_2"]=5
    data.head(10)
    return data

选择对象

In [ ]:
def dataset_generate(window_size,data,subject):
    # data_temp=data.loc[["Rebar2"],:]
    # data_temp=data.loc[data["Subject"]=="Rebar1" ,:]
    window=int(window_size*50)
    data_temp=data[data["Subject"].isin(subject)]
#     data_temp=data.loc[["Rebar2"],:]
#     data_temp=data.loc[data["Subject"]==subject ,:]
    data_subject=np.array(data_temp[data_temp["Label_2"].isin([1,2,3,4,5])])
    print(data_subject.shape)
    data_subject=np.reshape(data_subject,(-1,window,15))
    print(data_subject.shape)
    X=data_subject[:,:,3:]
    X=X.astype("float64")
    Y=data_subject[:,0,2]
    Y=Y.astype("int")
    Group=data_subject[:,0,1]
    print(X.shape)
    print(Y.shape)
    print(Group.shape)
    return X,Y,Group

查看数据类别分布

In [ ]:
from collections import Counter
def distribution_check(Y):
    Y_count=np.array(Y[:])
    print(Y.shape)
    print(Counter(Y_count))
#     print(Counter(Group))

整数编码

In [ ]:
from sklearn.preprocessing import LabelEncoder
def int_encoder(Y):
    label_encoder = LabelEncoder()
    Y = label_encoder.fit_transform(Y)
    print(Counter(Y))
    return Y

定义smote函数

In [ ]:
import imblearn as ib
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek 
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import CondensedNearestNeighbour
def smote(X_train,y_train,subject):
    dim0=X_train.shape[0]
    dim1=X_train.shape[1]
    dim2=X_train.shape[2]
    X_trian_for_smote=np.reshape(X_train,(dim0,dim1*dim2))
    
#     sme = SMOTETomek(random_state=42)
#     X_train_smo, y_train = sme.fit_resample(X_trian_for_smote, y_train)
#     rus = RandomUnderSampler(random_state=42,sampling_strategy='majority')
#     X_res, y_res = rus.fit_resample(X_trian_for_smote, y_train)
    if subject=="Carpenter1":
        smo = SMOTE(random_state=42,sampling_strategy={1:500,2:500})
    elif subject=="Carpenter2":
        smo = SMOTE(random_state=42,sampling_strategy={2:500})
#     elif subject=="Masonry1" or "Masonry2":
#         smo = SMOTE(random_state=42,sampling_strategy={1:300})
    elif subject=="Rebar1" or "Rebar2" or "Rebar3":
        smo = SMOTE(random_state=42,sampling_strategy={2:300,3:300})

    X_train_smo, y_train = smo.fit_resample(X_trian_for_smote, y_train)
#     cnn = CondensedNearestNeighbour(random_state=42,sampling_strategy='majority') 
#     X_res, y_train = cnn.fit_resample(X_trian_for_smote, y_train ) 
    X_train=np.reshape(X_train_smo,(X_train_smo.shape[0],dim1,dim2))
    return X_train,y_train

def smote_generalized(X_train,y_train,second):
    dim0=X_train.shape[0]
    dim1=X_train.shape[1]
    dim2=X_train.shape[2]
    X_trian_for_smote=np.reshape(X_train,(dim0,dim1*dim2))
    
    if second==0.6:
        smo = SMOTE(random_state=42,sampling_strategy={1:7000,2:7000,3:7000})
    elif second==0.8:
        smo = SMOTE(random_state=42,sampling_strategy={1:5000,2:5000,3:5000}) 
    elif second==1:
        smo = SMOTE(random_state=42,sampling_strategy={1:4000,2:4000,3:4000})  

    X_train_smo, y_train = smo.fit_resample(X_trian_for_smote, y_train)
#     cnn = CondensedNearestNeighbour(random_state=42,sampling_strategy='majority') 
#     X_res, y_train = cnn.fit_resample(X_trian_for_smote, y_train ) 
    X_train=np.reshape(X_train_smo,(X_train_smo.shape[0],dim1,dim2))
    return X_train,y_train

In [ ]:
from tensorflow.keras.backend import reshape
from tensorflow.keras import backend as k
class chan_attnlayer1(tf.keras.layers.Layer):
    def __init__ (self,num_channels=12,ratio=0.05):
        super(chan_attnlayer1,self).__init__()
        self.num_channels=num_channels
        self.ratio=ratio
        self.avg_pool=tf.keras.layers.GlobalAveragePooling2D(data_format="channels_last") #需要弄清楚channel是什么，在哪个维度上。
        self.max_pool=tf.keras.layers.GlobalMaxPool2D(data_format="channels_last")
        self.fc1=tf.keras.layers.Dense(num_channels//ratio,activation="relu")
        self.fc2=tf.keras.layers.Dense(num_channels,activation= None)
        self.softmax=tf.keras.layers.Activation('softmax')
    def get_config(self):
        config=super().get_config().copy()
        config.update({
            "num_channels":self.num_channels,
            "ratio":self.ratio
        })
        return config
    def call(self, inputs):
        self.inputs1=tf.transpose(inputs, [0,1,3,2])
#         print(self.inputs1.shape)
        self.avg_out=self.fc2(self.fc1(self.avg_pool(self.inputs1)))
#         print(self.avg_out.shape)
        self.max_out=self.fc2(self.fc1(self.max_pool(self.inputs1)))
#         print(self.max_out.shape)
        self.avg_out=tf.expand_dims(self.avg_out,1)
        self.avg_out=tf.expand_dims(self.avg_out,1)
        self.max_out=tf.expand_dims(self.max_out,1)
        self.max_out=tf.expand_dims(self.max_out,1)
#         print(self.avg_out.shape)
#         print(self.max_out.shape)
        self.out=self.softmax(self.avg_out+self.max_out)
#         self.out2=tf.multiply(self.inputs1,self.out)
#         self.out1=tf.transpose(self.out,[0,1,3,2])
#         print(self.out1.shape)
        self.out2=tf.squeeze(tf.matmul(self.inputs1,tf.transpose(self.out,[0,1,3,2])),axis=-1)
#         print(self.out2.shape)
#         b,h,w,c=k.shape(self.out2)
#         self.out3=tf.reshape(self.out,[-1,int(self.out2.shape[1]),int(self.out2.shape[2])*int(self.out2.shape[3])])
#         self.out3=tf.reshape(self.out2,[-1,self.out2.shape[1],self.out2.shape[2]*self.out2.shape[3]])
#         print(self.out2.shape)
        return self.out2

In [ ]:
class chan_attnlayer0(tf.keras.layers.Layer):
    def __init__ (self,num_channels=12,ratio=0.05):
        super(chan_attnlayer0,self).__init__()
        self.num_channels=num_channels
        self.ratio=ratio
        self.avg_pool=tf.keras.layers.GlobalAveragePooling2D(data_format="channels_last") #需要弄清楚channel是什么，在哪个维度上。
        self.max_pool=tf.keras.layers.GlobalMaxPool2D(data_format="channels_last")
        self.fc1=tf.keras.layers.Dense(num_channels//ratio,activation="relu")
        self.fc2=tf.keras.layers.Dense(num_channels,activation= None)
        self.sigmoid=tf.keras.layers.Activation('sigmoid')
    def get_config(self):
        config=super().get_config().copy()
        config.update({
            "num_channels":self.num_channels,
            "ratio":self.ratio
        })
        return config
    def call(self, inputs):
#         print(inputs.shape)
        self.inputs1=tf.transpose(inputs, [0,1,3,2])
#         self.inputs1=tf.transpose(inputs, [0,1,2,3])
#         print(self.inputs1.shape)
        self.avg_out=self.fc2(self.fc1(self.avg_pool(self.inputs1)))
#         print(self.avg_out.shape)
        self.max_out=self.fc2(self.fc1(self.max_pool(self.inputs1)))
#         print(self.max_out.shape)
        self.avg_out=tf.expand_dims(self.avg_out,1)
        self.avg_out=tf.expand_dims(self.avg_out,1)
        self.max_out=tf.expand_dims(self.max_out,1)
        self.max_out=tf.expand_dims(self.max_out,1)
#         print(self.avg_out.shape)
#         print(self.max_out.shape)
        self.out=self.sigmoid(self.avg_out+self.max_out)
#         print(self.out.shape)
#         self.out2=tf.multiply(self.inputs1,self.out)
#         self.out1=tf.transpose(self.out,[0,1,3,2])
#         print(self.out1.shape)
        self.out2=tf.transpose(tf.multiply(self.inputs1,self.out),[0,1,3,2])
#         print(self.out2.shape)
#         b,h,w,c=k.shape(self.out2)
#         self.out3=tf.reshape(self.out,[-1,int(self.out2.shape[1]),int(self.out2.shape[2])*int(self.out2.shape[3])])
#         self.out3=tf.reshape(self.out2,[-1,self.out2.shape[1],self.out2.shape[2]*self.out2.shape[3]])
#         print(self.out2.shape)
        return self.out2

In [ ]:
class chan_attnlayer2(tf.keras.layers.Layer):
    def __init__ (self,num_channels,ratio=16):
        super(chan_attnlayer2,self).__init__()
        self.num_channels=num_channels
        self.ratio=ratio
        self.avg_pool=tf.keras.layers.GlobalAveragePooling1D(data_format="channels_last") #需要弄清楚channel是什么，在哪个维度上。
        self.max_pool=tf.keras.layers.GlobalMaxPool1D(data_format="channels_last")
        self.fc1=tf.keras.layers.Dense(num_channels//ratio,activation="relu")
        self.fc2=tf.keras.layers.Dense(num_channels,activation= None)
#         self.fc1=tf.keras.layers.Conv2D(num_channels,num_channels//ratio,3,activation="relu")
#         self.fc2=tf.keras.layers.Conv2D(num_channels//ratio,num_channels,3,activation= None)
        self.sigmoid=tf.keras.layers.Activation('sigmoid')
    def get_config(self):
        config=super().get_config().copy()
        config.update({
            "num_channels":self.num_channels,
            "ratio":self.ratio
        })
        return config
    def call(self, inputs):
#         print(inputs.shape)#[30,256]
#         self.inputs1=tf.transpose(inputs, [0,1,3,2])
#         self.inputs1=tf.transpose(inputs, [0,1,2,3])
#         print(self.inputs1.shape)
        self.avg_out=self.fc2(self.fc1(self.avg_pool(inputs)))
#         print(self.avg_out.shape)
        self.max_out=self.fc2(self.fc1(self.max_pool(inputs)))
#         print(self.max_out.shape)
        self.avg_out=tf.expand_dims(self.avg_out,1)
#         self.avg_out=tf.expand_dims(self.avg_out,1)
#         self.max_out=tf.expand_dims(self.max_out,1)
        self.max_out=tf.expand_dims(self.max_out,1)
#         print(self.avg_out.shape)
#         print(self.max_out.shape)
        self.out=self.sigmoid(self.avg_out+self.max_out)
#         print(self.out.shape)
#         self.out2=tf.multiply(self.inputs1,self.out)
#         self.out1=tf.transpose(self.out,[0,1,3,2])
#         print(self.out1.shape)
        self.out2=tf.multiply(inputs,self.out)
#         print(self.out2.shape)
#         b,h,w,c=k.shape(self.out2)
#         self.out3=tf.reshape(self.out,[-1,int(self.out2.shape[1]),int(self.out2.shape[2])*int(self.out2.shape[3])])
#         self.out3=tf.reshape(self.out2,[-1,self.out2.shape[1],self.out2.shape[2]*self.out2.shape[3]])
#         print(self.out2.shape)
        return self.out2

In [ ]:
class chan_attnlayer3(tf.keras.layers.Layer):
    def __init__ (self,num_channels=256):
        super(chan_attnlayer3,self).__init__()
        self.num_channels=num_channels
#         self.fc1=tf.keras.layers.Dense(num_channels//ratio,activation="relu")
#         self.fc2=tf.keras.layers.Dense(num_channels,activation= None)
        self.fc1=tf.keras.layers.Conv2D(num_channels,3,dilation_rate=2,activation="relu", padding='same')
        self.fc2=tf.keras.layers.Conv2D(1,1,activation= None, padding='same')
        self.sigmoid=tf.keras.layers.Activation('softmax')
    def get_config(self):
        config=super().get_config().copy()
        config.update({
            "num_channels":self.num_channels
        })
        return config
    def call(self, inputs):
#         print(inputs.shape)#[30,256]
#         self.inputs1=tf.transpose(inputs, [0,1,3,2])
#         self.inputs1=tf.transpose(inputs, [0,1,2,3])
#         print(self.inputs1.shape)
#         self.avg_out=self.fc2(self.fc1(self.avg_pool(inputs)))
# #         print(self.avg_out.shape)
#         self.max_out=self.fc2(self.fc1(self.max_pool(inputs)))
#         print(self.max_out.shape)
#         self.avg_out=tf.expand_dims(self.avg_out,1)
# #         self.avg_out=tf.expand_dims(self.avg_out,1)
# #         self.max_out=tf.expand_dims(self.max_out,1)
#         self.max_out=tf.expand_dims(self.max_out,1)
#         print(self.avg_out.shape)
#         print(self.max_out.shape)
#         self.out=self.sigmoid(self.avg_out+self.max_out)
        self.inputs=tf.expand_dims(inputs,3)
#         print(self.inputs.shape)
        self.out=self.fc2(self.fc1(self.inputs))
#         print(self.out.shape)
#         print(self.avg_out.shape)
#         self.max_out=self.fc2(self.fc1(self.max_pool(inputs)))
#         print(self.out.shape)
#         self.out2=tf.multiply(self.inputs1,self.out)
#         self.out1=tf.transpose(self.out,[0,1,3,2])
#         print(self.out1.shape)
        self.out2=tf.multiply(inputs,tf.squeeze(self.out,axis=-1))
#         print(self.out2.shape)
#         b,h,w,c=k.shape(self.out2)
#         self.out3=tf.reshape(self.out,[-1,int(self.out2.shape[1]),int(self.out2.shape[2])*int(self.out2.shape[3])])
#         self.out3=tf.reshape(self.out2,[-1,self.out2.shape[1],self.out2.shape[2]*self.out2.shape[3]])
#         print(self.out2.shape)
        return self.out2

In [ ]:
#temporal_attention
class attnlayer(tf.keras.layers.Layer):
    def __init__(self, n_outputs=128):
        super(attnlayer, self).__init__()
        self.n_outputs = n_outputs
        self.fc1=tf.keras.layers.Dense(self.n_outputs,activation="tanh")
        self.fc2=tf.keras.layers.Dense(1,activation= "softmax")
        
#         self.activation1=activation1
#         self.activation2=activation2
#         self.tanh1=tf.keras.layers.Activation(activation1)
#         self.softmax1=tf.keras.layers.Activation(activation2)
#     def build(self):  # Create the state of the layer (weights)
#         self.w = self.add_weight(name='w',
#                                  shape=(64, self.n_outputs),
#                                  initializer='uniform',
#                                  trainable=True)
#         self.bias = self.add_weight(name='bias',
#                                  shape=(self.n_outputs,),
#                                  initializer='zero',
#                                  trainable=True)
#         self.w1 = self.add_weight(name='w1',
#                                  shape=(self.n_outputs, 1),
#                                  initializer='uniform',
#                                  trainable=True)
#         self.bias1 = self.add_weight(name='bias1',
#                                  shape=(1,),
#                                  initializer='zero',
#                                  trainable=True)
    def get_config(self):
        config=super().get_config().copy()
        config.update({
            "n_outputs":self.n_outputs
        })
        return config
    def call(self, inputs):  # Defines the computation from inputs to outputs
#         self.output1=self.tanh1(inputs @ self.w+self.bias)
#         self.score=self.softmax1(self.output1@self.w1+self.bias1)
        self.score=self.fc2(self.fc1(inputs))
#         print(self.score)
#         self.score1=self.score[1,:,:]
        self.output2=tf.matmul(tf.transpose(inputs, [0,2,1]),self.score)
#         print(self.output2.shape)
        
        return tf.squeeze(self.output2,axis=-1),self.score
        

构建模型

In [ ]:
# #LSTM
# from tensorflow.keras import Sequential
# from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
# # from tensorflow.keras.utils import plot_model
# from tensorflow.keras.layers import Conv2D, MaxPooling1D, LSTM,AveragePooling1D
# def construct_model(kernel_size,filters_num,pool,window_size_for_model,LSTM_num,class_num,conv_layer_num,learn_rate):
#     model = Sequential()
# #     for i in range(1,conv_layer_num+1):
# #         if i==1:
#     model.add(Conv2D(filters=filters_num, kernel_size=kernel_size, strides=(1,1),activation='relu',padding = 'same',input_shape=(window_size_for_model,12,1)))
# # #             model.add(Conv1D(filters=filters_num, kernel_size=kernel_size, strides=1,activation='relu',padding = 'same',input_shape=(window_size_for_model,12)))
# #             model.add(BatchNormalization())
            
# #             model.add(Conv2D(filters=filters_num, kernel_size=kernel_size, strides=(1,1), activation='relu',padding = 'same',input_shape=(window_size_for_model,12,1)))
# # #             model.add(Conv1D(filters=filters_num, kernel_size=kernel_size, strides=1,activation='relu',padding = 'same',input_shape=(window_size_for_model,12)))
# #             model.add(BatchNormalization())
                
# #             model.add(chan_attnlayer0())
# # #             model.add(AveragePooling1D(pool_size=pool))
# # #             model.add(MaxPooling1D(pool_size=pool))
# #         else:
# #             model.add(Conv2D(filters=filters_num, kernel_size=kernel_size, strides=(1,1), activation='relu',padding = 'same',input_shape=(window_size_for_model,12,1)))
# # #             model.add(Conv1D(filters=filters_num, kernel_size=kernel_size, strides=1,activation='relu',padding = 'same',input_shape=(window_size_for_model,12)))
# #             model.add(BatchNormalization())
            
# #             model.add(Conv2D(filters=filters_num, kernel_size=kernel_size, strides=(1,1), activation='relu',padding = 'same',input_shape=(window_size_for_model,12,1)))
# # #             model.add(Conv1D(filters=filters_num, kernel_size=kernel_size, strides=1,activation='relu',padding = 'same',input_shape=(window_size_for_model,12)))
# #             model.add(BatchNormalization())
# # #             model.add(chan_attnlayer0())
            
# #         Conv1D(filters=filters_num, kernel_size=kernel_size, activation='relu',padding = 'same', name = 'cnn_2'),
# # #         MaxPooling1D(pool_size=pool, name = 'maxpooling_2'),
# #         BatchNormalization(name='batchnorm_layer_2'),
# #         Conv1D(filters=filters_num, kernel_size=kernel_size, activation='relu',padding = 'same', name = 'cnn_3'),
# # #         MaxPooling1D(pool_size=pool, name = 'maxpooling_2'),
# #         BatchNormalization(name='batchnorm_layer_3'),
# #         Conv1D(filters=filters_num, kernel_size=kernel_size, activation='relu',padding = 'same', name = 'cnn_4'),
# # #         MaxPooling1D(pool_size=pool, name = 'maxpooling_2'),
# #         BatchNormalization(name='batchnorm_layer_4'),
# #         Conv1D(filters=filters_num, kernel_size=kernel_size, activation='relu',padding = 'same', name = 'cnn_5'),
# # #         MaxPooling1D(pool_size=pool, name = 'maxpooling_3'),
# #         BatchNormalization(name='batchnorm_layer_5'),
# #         Conv1D(filters=filters_num, kernel_size=kernel_size, activation='relu',padding = 'same', name = 'cnn_4'),
# # #         MaxPooling1D(pool_size=pool, name = 'maxpooling_3'),
# #         BatchNormalization(name='batchnorm_layer_4'),
# #         Conv1D(filters=filters_num, kernel_size=kernel_size, activation='relu',padding = 'same', name = 'cnn_5'),
# # #         MaxPooling1D(pool_size=pool, name = 'maxpooling_3'),
# #         BatchNormalization(name='batchnorm_layer_5'),
# #         Flatten(),
# #     model.add(chan_attnlayer1())
# #     model.add(tf.keras.Input(shape=(window_size_for_model,12)))
# #     model.add(Dense(256, activation = 'relu'))
#     model.add(tf.keras.layers.Reshape((30,-1)))
#     model.add(LSTM(LSTM_num,return_sequences = True, name = 'lstm_1'))
# #     model.add(LSTM(LSTM_num,input_shape=(window_size_for_model,filters_num),return_sequences = True, name = 'lstm_1'))
#     model.add(Dropout(0.5, name = 'Dropout_1'))
# #         BatchNormalization(name='batchnorm_layer_1'),
#     model.add(LSTM(LSTM_num, return_sequences = False, name = 'lstm_2'))
#     model.add(Dropout(0.5, name = 'Dropout_2'))
# #         LSTM(LSTM_num, return_sequences = False, name = 'lstm_3'),
# #         Dropout(0.4, name = 'Dropout_3'),
# #         BatchNormalization(name='batchnorm_layer_2'),
# #     model.add(Dense(256, activation = 'sigmoid'))
# #     model.add(Dense(256, activation = 'sigmoid'))
# #     model.add(Flatten())
# #     model.add(Dense(class_num, activation = 'softmax', name = 'output'))
# #     model.add(attnlayer())
#     #     Dense(128, activation= 'relu', name = 'dense_2'),
#     #     Dropout(0.3, name = 'Dropout_4'),
#     #     Dense(64, activation = 'relu', name = 'dense_3'),
#     #     Dropout(0.3, name = 'Dropout_5'),
# #         Dense(32, activation= 'relu', name = 'dense_1'),
#     #     Dropout(0.3, name = 'Dropout_3'),
#     model.add(Dense(class_num, activation = 'softmax', name = 'output'))

#     opt = tf.keras.optimizers.Adam(learning_rate=learn_rate)
# #     model.build(input_shape=(window_size_for_model,12))

#     model.compile(optimizer=opt, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
#     # plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
#     model.summary()
#     return model

In [ ]:
# #LSTM
# from tensorflow.keras import Sequential
# from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
# # from tensorflow.keras.utils import plot_model
# from tensorflow.keras.layers import Conv2D, MaxPooling1D, LSTM,AveragePooling1D
# def construct_model(kernel_size,filters_num,pool,window_size_for_model,LSTM_num,class_num,conv_layer_num,learn_rate):
#     model = Sequential()
# #     for i in range(1,conv_layer_num+1):
# #         if i==1:
# #             model.add(Conv2D(filters=filters_num, kernel_size=kernel_size, strides=(1,1),activation='relu',padding = 'same',input_shape=(window_size_for_model,12,1)))
# # #             model.add(Conv1D(filters=filters_num, kernel_size=kernel_size, strides=1,activation='relu',padding = 'same',input_shape=(window_size_for_model,12)))
# #             model.add(BatchNormalization())
            
# #             model.add(Conv2D(filters=filters_num, kernel_size=kernel_size, strides=(1,1), activation='relu',padding = 'same',input_shape=(window_size_for_model,12,1)))
# # #             model.add(Conv1D(filters=filters_num, kernel_size=kernel_size, strides=1,activation='relu',padding = 'same',input_shape=(window_size_for_model,12)))
# #             model.add(BatchNormalization())
                
# #             model.add(chan_attnlayer0())
# # #             model.add(AveragePooling1D(pool_size=pool))
# # #             model.add(MaxPooling1D(pool_size=pool))
# #         else:
# #             model.add(Conv2D(filters=filters_num, kernel_size=kernel_size, strides=(1,1), activation='relu',padding = 'same',input_shape=(window_size_for_model,12,1)))
# # #             model.add(Conv1D(filters=filters_num, kernel_size=kernel_size, strides=1,activation='relu',padding = 'same',input_shape=(window_size_for_model,12)))
# #             model.add(BatchNormalization())
            
# #             model.add(Conv2D(filters=filters_num, kernel_size=kernel_size, strides=(1,1), activation='relu',padding = 'same',input_shape=(window_size_for_model,12,1)))
# # #             model.add(Conv1D(filters=filters_num, kernel_size=kernel_size, strides=1,activation='relu',padding = 'same',input_shape=(window_size_for_model,12)))
# #             model.add(BatchNormalization())
# # #             model.add(chan_attnlayer0())
            
# #         Conv1D(filters=filters_num, kernel_size=kernel_size, activation='relu',padding = 'same', name = 'cnn_2'),
# # #         MaxPooling1D(pool_size=pool, name = 'maxpooling_2'),
# #         BatchNormalization(name='batchnorm_layer_2'),
# #         Conv1D(filters=filters_num, kernel_size=kernel_size, activation='relu',padding = 'same', name = 'cnn_3'),
# # #         MaxPooling1D(pool_size=pool, name = 'maxpooling_2'),
# #         BatchNormalization(name='batchnorm_layer_3'),
# #         Conv1D(filters=filters_num, kernel_size=kernel_size, activation='relu',padding = 'same', name = 'cnn_4'),
# # #         MaxPooling1D(pool_size=pool, name = 'maxpooling_2'),
# #         BatchNormalization(name='batchnorm_layer_4'),
# #         Conv1D(filters=filters_num, kernel_size=kernel_size, activation='relu',padding = 'same', name = 'cnn_5'),
# # #         MaxPooling1D(pool_size=pool, name = 'maxpooling_3'),
# #         BatchNormalization(name='batchnorm_layer_5'),
# #         Conv1D(filters=filters_num, kernel_size=kernel_size, activation='relu',padding = 'same', name = 'cnn_4'),
# # #         MaxPooling1D(pool_size=pool, name = 'maxpooling_3'),
# #         BatchNormalization(name='batchnorm_layer_4'),
# #         Conv1D(filters=filters_num, kernel_size=kernel_size, activation='relu',padding = 'same', name = 'cnn_5'),
# # #         MaxPooling1D(pool_size=pool, name = 'maxpooling_3'),
# #         BatchNormalization(name='batchnorm_layer_5'),
# #         Flatten(),
# #     model.add(chan_attnlayer1())
#     model.add(tf.keras.Input(shape=(window_size_for_model,12)))
#     model.add(Dense(256, activation = 'relu'))
#     model.add(LSTM(LSTM_num,input_shape=(window_size_for_model,filters_num),return_sequences = True, name = 'lstm_1'))
# #     model.add(LSTM(LSTM_num,input_shape=(window_size_for_model,filters_num),return_sequences = True, name = 'lstm_1'))
#     model.add(Dropout(0.4, name = 'Dropout_1'))
# #         BatchNormalization(name='batchnorm_layer_1'),
#     model.add(LSTM(LSTM_num, return_sequences = False, name = 'lstm_2'))
#     model.add(Dropout(0.4, name = 'Dropout_2'))
# #         LSTM(LSTM_num, return_sequences = False, name = 'lstm_3'),
# #         Dropout(0.4, name = 'Dropout_3'),
# #         BatchNormalization(name='batchnorm_layer_2'),
# #     model.add(Dense(256, activation = 'sigmoid'))
# #     model.add(Dense(256, activation = 'sigmoid'))
# #     model.add(Flatten())
# #     model.add(Dense(class_num, activation = 'softmax', name = 'output'))
# #     model.add(attnlayer())
#     #     Dense(128, activation= 'relu', name = 'dense_2'),
#     #     Dropout(0.3, name = 'Dropout_4'),
#     #     Dense(64, activation = 'relu', name = 'dense_3'),
#     #     Dropout(0.3, name = 'Dropout_5'),
# #         Dense(32, activation= 'relu', name = 'dense_1'),
#     #     Dropout(0.3, name = 'Dropout_3'),
#     model.add(Dense(class_num, activation = 'softmax', name = 'output'))

#     opt = tf.keras.optimizers.Adam(learning_rate=learn_rate)
# #     model.build(input_shape=(window_size_for_model,12))

#     model.compile(optimizer=opt, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
#     # plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
#     model.summary()
#     return model

In [ ]:
#1D卷积
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
# from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM,AveragePooling1D
def construct_model(kernel_size,filters_num,pool,window_size_for_model,LSTM_num,class_num,conv_layer_num,learn_rate):
    inputs=tf.keras.Input(shape=(window_size_for_model,12),name='img')
    
    h1=Conv1D(filters=filters_num, kernel_size=kernel_size, strides=1,padding = 'same')(inputs)
    h1=BatchNormalization()(h1)
    h1=tf.keras.layers.Activation("relu")(h1)
    h1=Conv1D(filters=filters_num, kernel_size=kernel_size, strides=1,padding = 'same')(h1)
    h1=BatchNormalization()(h1)
    
#     h1=chan_attnlayer2(64)(h1)
    h2=tf.keras.layers.Activation("relu")(h1)
    h2 = Conv1D(filters=filters_num, kernel_size=kernel_size, strides=1,padding = 'same')(h2)
    h2=BatchNormalization()(h2)
    h2=tf.keras.layers.Activation("relu")(h2)
    h2 = Conv1D(filters=filters_num, kernel_size=kernel_size, strides=1,padding = 'same')(h2)
    h2=BatchNormalization()(h2)
    block2_out = tf.keras.layers.add([h2, h1])

#     h2=chan_attnlayer2()(h2)
    
    
    
    h3=tf.keras.layers.Activation("relu")(block2_out)
    h3 = Conv1D(filters=filters_num, kernel_size=kernel_size, strides=1,padding = 'same')(h3)
    h3=BatchNormalization()(h3)
    h3=tf.keras.layers.Activation("relu")(h3)
    h3 = Conv1D(filters=filters_num, kernel_size=kernel_size, strides=1,padding = 'same')(h3)
    h3=BatchNormalization()(h3)
    block3_out = tf.keras.layers.add([h3, block2_out])
    h3=tf.keras.layers.Activation("relu")(block3_out)
#     h3=chan_attnlayer2()(h3)
#     block3_out = tf.keras.layers.add([h3, block2_out])
    
    h4=tf.keras.layers.Activation("relu")(h3)
    h4 = Conv1D(filters=filters_num, kernel_size=kernel_size, strides=1,padding = 'same')(h4)
    h4=BatchNormalization()(h4)
    h4=tf.keras.layers.Activation("relu")(h4)
    h4 = Conv1D(filters=filters_num, kernel_size=kernel_size, strides=1,padding = 'same')(h4)
    h4=BatchNormalization()(h4)
    block4_out = tf.keras.layers.add([h4, block3_out])
    h4=tf.keras.layers.Activation("relu")(block4_out)
    
    
    h5=LSTM(LSTM_num,input_shape=(window_size_for_model,filters_num),return_sequences = True, name = 'lstm_1')(h4)
    h5=tf.keras.layers.Dropout(0.5, name = 'Dropout_1')(h5)
    
    h6=LSTM(LSTM_num,input_shape=(window_size_for_model,filters_num),return_sequences = False, name = 'lstm_2')(h5)
    h6=tf.keras.layers.Dropout(0.5, name = 'Dropout_2')(h6)
#     h7=attnlayer()(h6)[0]
#     score=attnlayer()(h6)[1]
    
    outputs=tf.keras.layers.Dense(class_num, activation = 'softmax', name = 'output')(h6)
    
    model=tf.keras.Model(inputs,outputs,name="residual_CLN")

    opt = tf.keras.optimizers.Adam(learning_rate=learn_rate)

    model.compile(optimizer=opt, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    # plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
    model.summary()
    return model

模型训练

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from timeit import default_timer as timer
def model_fit(model,X_train, y_train,X_validation,y_validation,batch_size,epochs):
    earlystop_callback=EarlyStopping(monitor="val_loss",patience=20)
    modelcheckpoint=ModelCheckpoint(filepath="best_model.h5",monitor="val_accuracy",save_best_only=True)
    start = timer()
#     class_weight = {0 : 1., 1: 10., 2: 10., 3: 5., 4: 3.}

    history = model.fit( X_train, y_train, epochs = epochs,validation_data = (X_validation,y_validation), batch_size=batch_size ,callbacks=[earlystop_callback,modelcheckpoint], verbose=1)
#     history = model.fit( X_train, y_train, epochs = 200,validation_data = (X_validation,y_validation), batch_size=batch_size , verbose=1)
    #     validation_data = (X_validation,y_validation)

    end = timer()
    print("\n")
    print("Time: ",(end - start),"secs = ",(end - start)/3600,"hours")


    # ## EVALUATION AND THE PERFORMANCE METRICS

    # In[62]:


    # Plotting loss and accuracy graph
    plt.figure(figsize=(6, 4))
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)

    plt.plot(np.array(history.history['loss']), "r--", label="Train loss")
    plt.plot(np.array(history.history['accuracy']), "g--", label="Train accuracy")

    plt.plot(np.array(history.history['val_loss']), "r-", label="validation loss")
    plt.plot(np.array(history.history['val_accuracy']), "g-", label="validation accuracy")

    plt.title("Training session's progress over iterations",fontsize=10)
    plt.legend(loc='best', shadow=True,fontsize=10)
    plt.ylabel('Training Progress (Loss or Accuracy values)',fontsize=10)
    plt.xlabel('Training Epoch',fontsize=10)
    plt.ylim(0)

    plt.show()

绘图

In [ ]:
import itertools
from sklearn.metrics import confusion_matrix
plt.rcParams["font.family"] = 'DejaVu Sans'
def plot_confusion_matrix(cm, classes,class_num,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title,fontsize=20)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0,fontsize=25)
    plt.yticks(tick_marks, classes,fontsize=25)
    ax = plt.gca()
    ax.set_ylim(-.5, class_num-0.5)
    ax.set_xlim(-.5, class_num-0.5)
        
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",fontsize=20,
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label',fontsize=20)
    plt.xlabel('Predicted label',fontsize=20)

模型预测以及结果输出

In [ ]:
def model_test(model,X_test,y_test,class_num,LABELS):
#     results = dict()
    test_pred = np.argmax(model.predict(X_test), axis=1)
#     print(test_pred)
    #输出分类结果
#     print(y_test)
#     print(test_pred)
    classfication_report=metrics.classification_report(y_test,test_pred)
    results=float(100*metrics.f1_score(y_test, test_pred, average="macro"))
#     results['classification_report'] = classification_report
    print(classfication_report)
    #输出混淆矩阵
    cm = metrics.confusion_matrix(y_test, test_pred)
    print("Confusion Matrix:")
    print(cm)
    #绘制归一化混淆矩阵
    plt.figure(figsize=(class_num,class_num))
    plt.grid(b=False)
    plot_confusion_matrix(cm, LABELS,class_num, normalize=True, title='Normalized confusion matrix',cmap=plt.cm.Reds)
    ax = plt.gca()
    plt.show()
    #绘制非归一化混淆矩阵
    plt.figure(figsize=(class_num,class_num))
    plot_confusion_matrix(cm, LABELS,class_num, normalize=False, title='confusion matrix',cmap=plt.cm.Reds)
    ax = plt.gca()
    plt.show()
    return results,cm

In [ ]:
# from sklearn.model_selection import StratifiedKFold
# skf1 = StratifiedKFold(n_splits=5,shuffle=True)
# skf2 = StratifiedKFold(n_splits=5,shuffle=False)

# from sklearn.model_selection import LeaveOneGroupOut
# logo = LeaveOneGroupOut()
# # logo.get_n_splits(X, Y, group)

# from sklearn.model_selection import LeavePGroupsOut
# lpgo = LeavePGroupsOut(n_groups=2)

# from sklearn.model_selection import StratifiedGroupKFold
# sgkf=StratifiedGroupKFold(n_splits=3,shuffle=True)

# window_sizes=[0.6]
# positions=["xd"]
# conv_layers_num=[2] #最优2
# filters_nums=[256] #最优256
# kernel_sizes=[3] #最优3
# LSTM_nums=[64] #最优64

# for position in positions:
#     for window_size in window_sizes:
#         for conv_layer_num in conv_layers_num:
#             for filters_num in filters_nums:
#                 for kernel_size in kernel_sizes:
#                     for LSTM_num in LSTM_nums:
#                         print("--------------开始"+position+"_"+str(window_size)+"s_卷积层数为"+str(conv_layer_num)+"_lstm单元数为"+str(LSTM_num)+"_卷积核数为"+str(filters_num)+"_卷积核大小为"+str(kernel_size)+"的personalized实验--------------")
#                         data=load_data(window_size,position)
#                         class_num=5
# #                         subjects=["Rebar1","Rebar2","Rebar3"]
# #                         subjects=["Rebar1","Rebar2","Rebar3","Carpenter1","Carpenter2","Masonry1","Masonry2"]
#                         subjects=["Rebar1","Rebar2","Rebar3"]
#                         LABELS = [ "ST", "WK",  "TR", "BD", "SQ"]
#                 #         for subject in subjects:
#                 #             if subject=="Carpenter1" or subject=="Carpenter2":
#                 #                 class_num=5
#                 #                 LABELS = [ "ST", "WK",  "TR", "BD", "SQ"] 
#                 # #             elif subject=="Masonry1" or subject=="Masonry2":
#                 # #                 class_num=6
#                 # #                 LABELS = [ "ST", "WK",  "TR", "BD", "SQ","WO"] 
#                 #             else:
#                 #                 LABELS = [ "ST", "WK", "TR", "BD", "SQ"]

#                 #         print("--------------开始第"+subject+"--------------")
#                         X,Y,Group=dataset_generate(window_size,data,subjects)
#                         distribution_check(Y)
#                         Y=int_encoder(Y)
#                         # filter=[32,64,128]
#                         # kernel_sizes=[3,4,7,9]
#                         # LSTM_nums=[16,32,64,128]
#                         # for kernel_size in kernel_sizes:
#                         average_macrof1_score_total=0.0
#                         num=1
#                         confusion_matrix_total=np.zeros((class_num,class_num))
#                         # print("--------------开始LSTM_num为"+str(kernel_size)+"的实验--------------")
# #                         for train_total_index, test_index in skf1.split(X, Y):
#                         for train_total_index, test_index in logo.split(X, Y,Group):
# #                             print("--------------开始第"+str(num)+"次交叉验证--------------")
#                             print("当前为"+str(Group[test_index[0],]))
                            
#                             confusion_matrix_subject=np.zeros((class_num,class_num))
#                             average_macrof1_score_subject=0.0
            
#                             X_all, y_all = X[test_index],Y[test_index]
#                             print(Counter(y_all))
#                             X_train, y_train = [],[]
#                             X_validation,y_validation=[],[]
#                             number=1
#                             for train_index, validation_index in skf1.split(X_all, y_all):
#                                 X_train, y_train = X_all[train_index],y_all[train_index]
#                                 print(Counter(y_train))
#                                 X_train, y_train=smote(X_train, y_train,str(Group[test_index[0],]))
#                                 X_validation,y_validation=X_all[validation_index],y_all[validation_index]
#                                 print("训练集第"+str(number)+"次")
#                                 print("训练集比例")
#                                 print(Counter(y_train))
#                                 print("测试集比例")
#                                 print(Counter(y_validation))
#                                 model=construct_model(conv_layer_num=conv_layer_num ,kernel_size=kernel_size,filters_num=filters_num,pool=3,window_size_for_model=int(window_size*50),LSTM_num=LSTM_num,class_num=class_num)
#                                 model_fit(model,X_train=X_train,y_train=y_train,X_validation=X_validation,y_validation=y_validation,batch_size=64)
#                                 results,confusion_matrix_temp=model_test(model,X_test=X_validation,y_test=y_validation,class_num=class_num,LABELS=LABELS)
#                                 average_macrof1_score_subject=average_macrof1_score_subject + results
#                                 confusion_matrix_subject=confusion_matrix_subject+confusion_matrix_temp
#                                 del model
#                                 number=number+1
#                             print(confusion_matrix_subject)
#                             print(str(Group[test_index[0],])+"_"+position+"_"+str(window_size)+"的average-macro-f1-score:"+str(average_macrof1_score_subject/(number-1))+"%")
#                             #         break
#                             num=num+1
# #                             break

#                 #             plt.figure(figsize=(5,5))
#                 #             plt.grid(b=False)
#                 #             plot_confusion_matrix(confusion_matrix, classes=LABELS, normalize=True, title='Normalized confusion matrix',cmap=plt.cm.Reds)
#                 #             ax = plt.gca()
#                 #             plt.show()
#                 #             plt.figure(figsize=(5,5))
#                 #             plot_confusion_matrix(confusion_matrix, classes=LABELS, normalize=False, title='confusion matrix',cmap=plt.cm.Reds)
#                 #             ax = plt.gca()
#                 #             plt.show()
#                         print(confusion_matrix_total)
#                         print(position+"_"+str(window_size)+"的average-macro-f1-score:"+str(average_macrof1_score_total/(num-1))+"%")

SRS划分数据集(generalized model)

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf1 = StratifiedKFold(n_splits=5,shuffle=True)
skf2 = StratifiedKFold(n_splits=5,shuffle=True)

from sklearn.model_selection import LeaveOneGroupOut
logo = LeaveOneGroupOut()
# logo.get_n_splits(X, Y, group)

from sklearn.model_selection import LeavePGroupsOut
lpgo = LeavePGroupsOut(n_groups=2)

from sklearn.model_selection import StratifiedGroupKFold
sgkf=StratifiedGroupKFold(n_splits=3,shuffle=True)

window_sizes=[0.8]
positions=["kd","xd"]
conv_layers_num=[2] #最优2
filters_nums=[256] #最优256
kernel_sizes=[3]#最优3(4,1)
LSTM_nums=[64] #最优64


for position in positions:
    for window_size in window_sizes:
        for conv_layer_num in conv_layers_num:
            for filters_num in filters_nums:
                for kernel_size in kernel_sizes:
                    for LSTM_num in LSTM_nums:
                        print("--------------开始"+position+"_"+str(window_size)+"s_卷积层数为"+str(conv_layer_num)+"_lstm单元数为"+str(LSTM_num)+"_卷积核数为"+str(filters_num)+"_卷积核大小为"+str(kernel_size)+"的generalized实验--------------")
                        data=load_data(window_size,position)
                        class_num=5
#                         subjects=["Rebar1","Rebar2","Rebar3"]
                        subject_names=["Rebar1","Rebar2","Rebar3","Carpenter1","Carpenter2","Masonry1","Masonry2"]
#                         subject_names=["David","WGH","FL","RQB","WHT","FX","YJJ","GRH"]
#                         subjects=["Rebar1","Rebar2","Rebar3"]
                        LABELS = [ "1", "2",  "3", "4", "5" ]
                #         for subject in subjects:
                #             if subject=="Carpenter1" or subject=="Carpenter2":
                #                 class_num=5
                #                 LABELS = [ "ST", "WK",  "TR", "BD", "SQ"] 
                # #             elif subject=="Masonry1" or subject=="Masonry2":
                # #                 class_num=6
                # #                 LABELS = [ "ST", "WK",  "TR", "BD", "SQ","WO"] 
                #             else:
                #                 LABELS = [ "ST", "WK", "TR", "BD", "SQ"]

                #         print("--------------开始第"+subject+"--------------")
                        X,Y,Group=dataset_generate(window_size,data,subject_names)
                        distribution_check(Y)
                        Y=int_encoder(Y)
                        # filter=[32,64,128]
                        # kernel_sizes=[3,4,7,9]
                        # LSTM_nums=[16,32,64,128]
                        # for kernel_size in kernel_sizes:
                        average_macrof1_score=0.0
                        average_accuracy=0.0
                        num=1
                        confusion_matrix=np.zeros((class_num,class_num))
                        # print("--------------开始LSTM_num为"+str(kernel_size)+"的实验--------------")
                        for train_total_index, test_index in skf1.split(X, Y):
#                         for train_total_index, test_index in logo.split(X, Y,Group):
#                             print("--------------开始第"+str(num)+"次交叉验证--------------")
#                             print("当前测试集为"+str(Group[test_index[0],]))
#                             if str(Group[test_index[0],]) != "Masonry1"  : 
#                                 continue
                        #     print("TRAIN:", train_index.shape, "TEST:", test_index.shape)
                        #     X_train, X_test = X[train_index], X[test_index]
                        #     y_train, y_test = Y[train_index], Y[test_index]
                        #     print(train_total_index)
                            X_test, y_test = X[test_index],Y[test_index]
                            X_train_total,y_train_total=X[train_total_index],Y[train_total_index]
                            print(Counter(y_train_total))
                            X_train_total,y_train_total=smote_generalized(X_train_total,y_train_total,window_size)
                            print(Counter(y_train_total))
                            X_train, y_train = [],[]
                            X_validation,y_validation=[],[]
                            print(y_train_total)
                            number=1
                            for train_index, validation_index in skf1.split(X_train_total, y_train_total):
                                if number==1:
                                    X_train, y_train = X_train_total[train_index],y_train_total[train_index]
                                    X_validation,y_validation=X_train_total[validation_index],y_train_total[validation_index]
                                    number=number+1
                                else:
                                    break
                        #     Group_train_total=Group[train_total_index]
#                             print(train_total_index.shape)
#                             print(test_index.shape)
#                             print(Counter(y_test))
#                             print(Counter(y_train)) 
#                             print(Counter(y_validation))
                        #     print(Counter(y_train_total))
                        #     for train_index,validation_index in skf1.split(X_train_total,y_train_total):
                        # #         print("当前验证集为"+str(Group_train_total[validation_index[0],]))
                        #         X_train, X_validation = X_train_total[train_index], X_train_total[validation_index]
                        #         y_train, y_validation = y_train_total[train_index], y_train_total[validation_index]
                        #         print(train_index.shape)
                        #         print(validation_index.shape)

                        #         print(Counter(y_train))
                        #         print(Counter(y_validation))
        #                     smote(X_train_total,y_train_total)
        
        
#                             X_train=tf.expand_dims(X_train,-1)
#                             X_validation=tf.expand_dims(X_validation,-1)
#                             X_test=tf.expand_dims(X_test,-1)
                            print(X_train.shape,X_validation.shape,X_test.shape)
                    
                    
#                             print("输入维度")
#                             print(X_train.shape)
                            model=construct_model(conv_layer_num=conv_layer_num ,kernel_size=kernel_size,filters_num=filters_num,pool=3,window_size_for_model=int(window_size*50),LSTM_num=LSTM_num,class_num=class_num,learn_rate=0.00005)
                            model_fit(model,X_train=X_train,y_train=y_train,X_validation=X_validation,y_validation= y_validation,batch_size=256,epochs=300)
                            results,confusion_matrix_temp=model_test(model,X_test=X_test,y_test=y_test,class_num=class_num,LABELS=LABELS)
                        #     print(results['f1-score'])
                            average_macrof1_score=average_macrof1_score + results
                            confusion_matrix=confusion_matrix+confusion_matrix_temp
                            del model
                        #         break
                            num=num+1
                            break

                #             plt.figure(figsize=(5,5))
                #             plt.grid(b=False)
                #             plot_confusion_matrix(confusion_matrix, classes=LABELS, normalize=True, title='Normalized confusion matrix',cmap=plt.cm.Reds)
                #             ax = plt.gca()
                #             plt.show()
                #             plt.figure(figsize=(5,5))
                #             plot_confusion_matrix(confusion_matrix, classes=LABELS, normalize=False, title='confusion matrix',cmap=plt.cm.Reds)
                #             ax = plt.gca()
                #             plt.show()
                        print(confusion_matrix)
                        print(position+"_"+str(window_size)+"的average-macro-f1-score:"+str(average_macrof1_score/(num-1))+"%")

